In [22]:
import csv
import cv2
import numpy as np

lines=[]
with open('./Clone/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
adjusted_angle = 0.2

for line in lines:
    measurement = float(line[3])
    
    center_image = cv2.imread('./Clone/IMG/' + line[0].split('\\')[-1])
    images.append(center_image)
    measurements.append(measurement)
    
    left_image = cv2.imread('./Clone/IMG/' + line[1].split('\\')[-1])
    images.append(left_image)
    measurements.append(measurement + adjusted_angle)
    
    right_image = cv2.imread('./Clone/IMG/' + line[2].split('\\')[-1])
    images.append(right_image)
    measurements.append(measurement - adjusted_angle)   
    


In [23]:
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    #augmented_images.append(image)
    #augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)

images.extend(augmented_images)
measurements.extend(augmented_measurements)
    

In [24]:
print(len(measurements))

50634


In [25]:
X_train = np.array(images)
y_train = np.array(measurements)

In [26]:
from keras.models import Sequential, Model
from keras.layers import Activation, Flatten, Dense, Lambda, Cropping2D, Convolution2D
from keras import backend as K
import matplotlib.pyplot as plt


model = Sequential()
model.add(Lambda(lambda x: x /255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20),(0,0))))
model.add(Convolution2D(24,(5,5),strides=(2,2),activation="relu"))
model.add(Convolution2D(36,(5,5),strides=(2,2),activation="relu"))
model.add(Convolution2D(48,(5,5),strides=(2,2),activation="relu"))
model.add(Convolution2D(64,(3,3),activation="relu"))
model.add(Convolution2D(64,(3,3),activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
#model.add(Activation('softmax'))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=3)

model.save('model.h5')


Train on 40507 samples, validate on 10127 samples
Epoch 1/3
 1632/40507 [>.............................] - ETA: 19:20 - loss: 0.1097

KeyboardInterrupt: 